# The Ozone Hole
Ruth Moorman May 14th 2024 (for Nathaniel Livesey ESE144 lectures)



<div class="alert alert-block alert-info">
<b>DATA:</b>
<div> 
<div>        
    
I unfortunately had difficulty obtaining ozone data derived from the Microwave Limb Sounder for out activity. A gridded product looks like it should be available <a href="https://search.earthdata.nasa.gov/search/granules/collection-details?p=C2042566088-GES_DISC&pg%5B0%5D%5Bv%5D=f&pg%5B0%5D%5Bgsk%5D=-start_date&g=G3003268466-GES_DISC&q=ozone&fi=MLS&fl=3%2B-%2BGridded%2BObservations&tl=1715708126.667%213%21%21&zoom=0">here</a> but when I downloaded data from this source it did not have the structure I anticipated (think there may be an issue here...). So here I'm using some L4 model (data assimilating) output, since it has units of Dobson Units, from, you guessed it, <a href="https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-ozone-v1?tab=overview">Copernicus</a>.  
    
<div>
<div>

    
We have a monthly gridded timeseries of total column ozone from 1979 to 2018.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import matplotlib.ticker as mticker
import warnings
warnings.filterwarnings('ignore')


In [ ]:
MSR_interpolated_ozone = xr.open_mfdataset('~/shared/notebooks/CaltechESE1442024/data/ozonefulltimeseries/*.nc').sel(latitude = slice(-90,-30))
total_ozone_column     = MSR_interpolated_ozone.total_ozone_column
total_ozone_column

In [ ]:
longitude = total_ozone_column.longitude
latitude  = total_ozone_column.latitude

## Plot total column ozone for the year of the largest ozone hole (2006)

In [ ]:
dobsonunit_2006 = total_ozone_column.sel(time='2006')

In [ ]:
dobsonunit_2006

In [ ]:
fig, axs = plt.subplots(4,3,figsize=(15, 20), subplot_kw={'projection': ccrs.SouthPolarStereo()},gridspec_kw={'wspace':0.1,'hspace':0.1})
for row in range(4):
    for column in range(3):
        axs[row,column].add_feature(cartopy.feature.COASTLINE)
        theta = np.linspace(0, 2*np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)
        axs[row,column].set_boundary(circle, transform=axs[row,column].transAxes)
        axs[row,column].set_extent([-180, 180, -90, -30], ccrs.PlateCarree())

vmin=200
vmax=320
c = axs[0,0].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=0),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[0,1].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=1),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[0,2].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=2),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())

axs[1,0].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=3),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[1,1].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=4),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[1,2].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=5),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())

axs[2,0].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=6),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[2,1].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=7),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[2,2].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=8),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())

axs[3,0].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=9),vmin=vmin,  vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[3,1].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=10),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())
axs[3,2].pcolormesh(longitude,latitude,dobsonunit_2006.isel(time=11),vmin=vmin, vmax=vmax,cmap='gist_ncar',zorder=1, transform=ccrs.PlateCarree())

axs[0,0].set_title('Jan')
axs[0,1].set_title('Feb')
axs[0,2].set_title('Mar')

axs[1,0].set_title('Apr')
axs[1,1].set_title('May')
axs[1,2].set_title('Jun')

axs[2,0].set_title('Jul')
axs[2,1].set_title('Aug')
axs[2,2].set_title('Sept')

axs[3,0].set_title('Oct')
axs[3,1].set_title('Nov')
axs[3,2].set_title('Dec')

cax = fig.add_axes([0.25, 0.93, 0.5, 0.015]); cbar=plt.colorbar(c,cax = cax,orientation='horizontal'); cbar.set_label('DU', fontsize = 12)

plt.show()

## Timeseries of Ozone Hole Area

Ozone hole area is typically defined as the total area at a given time in the southern hemisphere with total column ozone less than 220 Dobson Units.

To calculate this, we'll need to generate a cell area array (again, similar to last week, this will be very approximate here since we're missing some metadata...) and use the `xarray.where()` function.

Note, this will be coming from L4 monthly mean data, so won't capture extreme values on smaller timescales.

In [ ]:
latitude.values

In [ ]:
## have to assume a central grid
lat_lower = np.append(-90,latitude[1:] - np.diff(latitude)/2)
lat_upper = np.append(-89.75,latitude[1:] + np.diff(latitude)/2)
R = 6378 * 1e3
sinϕ1_minus_sinϕ2   = np.sin(np.deg2rad(lat_lower)) - np.sin(np.deg2rad(lat_upper)) # one dimesional array
_,sinϕ1_minus_sinϕ2 = np.meshgrid(np.zeros(720),sinϕ1_minus_sinϕ2) # make into a grid
θ1_minus_θ2         = np.deg2rad(-179.75) - np.deg2rad(-179.25) ## this is constant for this grid
cell_area           = R**2 * sinϕ1_minus_sinϕ2 * θ1_minus_θ2
cell_area           = xr.DataArray(cell_area, coords = [dobsonunit_2006.latitude,dobsonunit_2006.longitude ], dims=['latitude', 'longitude'])

In [ ]:
## stack in time
cell_area_full = cell_area.expand_dims({'time':total_ozone_column.time})


<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>   
    Calculate and plot a timeseries of ozone hole area in millions of km2 (m2/1e12). [ use where.() function]
<div>
<div>

    



<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>        
    
Plot the same using clear sky values (rlut_clear and rsut_clear).<div><div>

    





<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>   
Plot the maximum (monthly) ozone hole area for each year. [use groupby('time.year')] <div>
<div>

    
